In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, Add, Average
from tensorflow.keras.metrics import Recall,Precision
from sklearn.metrics import f1_score
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # tensorflow학습시 warning 숨김
import warnings
warnings.filterwarnings(action='ignore') # 그래프 그릴때 경고 메세지 안 보이게
plt.rc('font', family = 'AppleGothic')

In [17]:
asos_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_1980_2024_After_drop.csv')

In [18]:
label_encoding_dict = {
    "no_clouds": 0,
    "Sc": 1,
    "Ci": 2,
    "ScAs": 3,
    "StNs": 4,
    "Ac": 5,
    "As": 6,
    "ScAc": 7,
    "ScCi": 8,
    "CuSc": 9,
    "AcCi": 10,
    "StAs": 11,
    "StSc": 12,
    "St": 13,
    "Cu": 14,
    "Cs": 15,
    "CuCi": 16,
    "ScNs": 17,
    "AsCi": 18,
    "CuAc": 19,
    "StCi": 20,
    "AcCs": 21,
    "ScCs": 22,
    "Cc": 23,
    "ScAcCi": 24,
    "CuAs": 25,
    "CsCi": 26,
    "AsAc": 27,
    "CbSt": 28,
    "CuSt": 29,
    "CbStNs": 30,
    "AcCc": 31,
    "AsCs": 32,
    "CbSc": 33,
    "CbCu": 34,
    "CbNs": 35,
    "Ns": 36,
    "ScCc": 37,
    "CuScCi": 38,
    "StAc": 39,
    "CcCi": 40,
    "CuScAs": 41,
    "CbAs": 42,
    "CuScAc": 43,
    "ScAcCs": 44,
    "Cb": 45,
    "CuScAcCi": 46,
    "CuScCc": 47,
    "CuCs": 48,
    "CbCi": 49,
    "CuAcCi": 50,
    "CuCc": 51,
    "ScAcCc": 52,
    "CuScCs": 53,
    "AsCc": 54,
    "ScCcCi": 55,
    "CbAc": 56,
    "CbCuNs": 57,
    "CuNs": 58,
    "CuScAsCi": 59,
    "ScAsCi": 60,
    "AcCcCi": 61,
    "CuScNs": 62,
    "AsSc": 63,
    "ScCu": 64,
    "CuScAcCc": 65,
    "CbScNs": 66,
    "CuScCcCi": 67,
    "CbScCs": 68,
    "CuCcCi": 69,
    "CuAcCc": 70,
    "StAsCi": 71,
    "CbScAs": 72,
    "ScAsCs": 73
}

In [19]:
asos_df['clfmAbbrCd'] = asos_df['clfmAbbrCd'].map(lambda x: label_encoding_dict[x])

In [20]:
asos_df['group'] = np.arange(len(asos_df)) // 24

In [21]:
asos_df

,tm,hm,pv,td,pa,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,rain,group
0,1980-01-01 01:00:00,NaN,3.700757,NaN,1013.876282,0,0.0,NaN,NaN,NaN,NaN,0,0
1,1980-01-01 02:00:00,NaN,3.700757,NaN,1013.876282,0,0.0,NaN,NaN,NaN,NaN,0,0
2,1980-01-01 03:00:00,56.000000,3.900000,-6.0,1015.900000,2,0.0,-1.200000,0.300000,0.500000,0.6,0,0
3,1980-01-01 04:00:00,58.333333,3.700757,-5.6,1013.876282,0,0.0,-1.083333,0.316667,0.483333,0.6,0,0
4,1980-01-01 05:00:00,60.666667,3.700757,-5.2,1013.876282,0,0.0,-0.966667,0.333333,0.466667,0.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127389,2024-12-30 21:00:00,67.000000,6.100000,0.0,1005.400000,2,36.5,-0.100000,-0.300000,0.800000,1.7,0,5307
127390,2024-12-30 22:00:00,71.000000,6.100000,0.0,1005.600000,0,0.0,-0.100000,-0.300000,0.800000,1.7,0,5307
127391,2024-12-30 23:00:00,75.000000,6.100000,0.0,1005.400000,0,0.0,-0.100000,-0.300000,0.800000,1.7,0,5307
127392,2024-12-31 00:00:00,81.000000,7.000000,1.8,1005.200000,0,0.0,-0.100000,-0.300000,0.800000,1.7,0,5308


In [22]:
def time_rain_whether_LSTM(time):
    return asos_df['rain'].shift(-time)

In [23]:
df_3h = asos_df.copy()
df_6h = asos_df.copy()
df_12h = asos_df.copy()
df_24h = asos_df.copy()

In [24]:
df_3h['hours_3'] = time_rain_whether_LSTM(3)
df_6h['hours_6'] = time_rain_whether_LSTM(6)
df_12h['hours_12'] = time_rain_whether_LSTM(12)
df_24h['hours_24'] = time_rain_whether_LSTM(24)

In [40]:
def processing_data(data, hours_col):
    data.drop(index=data[data.iloc[:,-1].isna()].index, inplace=True)
    group_sums = data.groupby('group')[hours_col].sum()
    groups_to_delete = group_sums[group_sums == 0].index
    groups_to_delete = groups_to_delete[:2500]
    filtered_df = data[~data['group'].isin(groups_to_delete)]
    other_columns_grouped = filtered_df.groupby('group')[['hm', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'clfmAbbrCd', 'lcsCh']].apply(lambda x: x.values.tolist())
    hours_grouped = filtered_df.groupby('group')[hours_col].apply(list)
    y_data = []
    for hours in hours_grouped:
        if 1 in hours:
            y_data.append(1)
        else:
            y_data.append(0)
    data_X = np.vstack(other_columns_grouped.iloc[:-1].tolist())
#     print(data_X.shape)
    data_y = np.array(y_data[:-1])
#     print(data_y.shape)
    scaler = MinMaxScaler()
    data_X = scaler.fit_transform(data_X.reshape(-1,1)).reshape(-1,24,9)
    spilt_index = round(len(data_X)/10)*7
    train_X = data_X[:spilt_index]
    train_y = data_y[:spilt_index]
    test_X = data_X[spilt_index:]
    test_y = data_y[spilt_index:]
    return train_X,train_y,test_X,test_y

In [42]:
train_3h_X,train_3h_y,test_3h_X,test_3h_y = processing_data(df_3h,'hours_3')
train_6h_X,train_6h_y,test_6h_X,test_6h_y = processing_data(df_6h,'hours_6')
train_12h_X,train_12h_y,test_12h_X,test_12h_y = processing_data(df_12h,'hours_12')
train_24h_X,train_24h_y,test_24h_X,test_24h_y = processing_data(df_24h,'hours_24')

In [46]:
print(train_3h_X.shape,train_3h_y.shape,test_3h_X.shape,test_3h_y.shape)
print(train_6h_X.shape,train_6h_y.shape,test_6h_X.shape,test_6h_y.shape)
print(train_12h_X.shape,train_12h_y.shape,test_12h_X.shape,test_12h_y.shape)
print(train_24h_X.shape,train_24h_y.shape,test_24h_X.shape,test_24h_y.shape)

(1967, 24, 9) (1967,) (840, 24, 9) (840,)
(1967, 24, 9) (1967,) (840, 24, 9) (840,)
(1967, 24, 9) (1967,) (840, 24, 9) (840,)
(1967, 24, 9) (1967,) (840, 24, 9) (840,)


In [50]:
def lstm_model_V2():

    inputs = Input(shape=(24, 9))

    layer_1 = LSTM(units=32, return_sequences=True)(inputs)
    
    layer_2_1 = LSTM(units=32, return_sequences=False,
                     kernel_initializer='glorot_uniform',  # 가중치 초기화
                     recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                     bias_initializer='zeros')(layer_1)
    layer_2_2 = LSTM(units=32, return_sequences=False,
                     kernel_initializer='glorot_uniform',  # 가중치 초기화
                     recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                     bias_initializer='zeros')(inputs)
    second_layer = Add()([layer_2_1, layer_2_2])
    dropout_1 = Dropout(0.2)(second_layer)
    
    D_1 = Dense(units=128, activation='relu')(dropout_1)
    dropout_2 = Dropout(0.2)(D_1)
    D_2 = Dense(units=256, activation='relu')(dropout_2)
    dropout_3 = Dropout(0.2)(D_2)
    D_3 = Dense(units=128, activation='relu')(dropout_3)
    dropout_4 = Dropout(0.2)(D_3)
    D_4 = Dense(units=64, activation='relu')(dropout_4)
    dropout_5 = Dropout(0.2)(D_4)
    D_5 = Dense(units=32, activation='relu')(dropout_5)
    dropout_6 = Dropout(0.2)(D_5)
    
    outputs = Dense(units=1, activation='sigmoid')(dropout_6)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',Recall(),Precision()])
    return model

In [51]:
model = lstm_model_V2()

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 24, 9)]      0           []                               
                                                                                                  
 lstm_3 (LSTM)                  (None, 24, 32)       5376        ['input_2[0][0]']                
                                                                                                  
 lstm_4 (LSTM)                  (None, 32)           8320        ['lstm_3[0][0]']                 
                                                                                                  
 lstm_5 (LSTM)                  (None, 32)           5376        ['input_2[0][0]']                
                                                                                            

In [52]:
model.fit(train_12h_X,train_12h_y, validation_split=0.2, epochs=50)

Epoch 1/50
50/50 [==============================] - 10s 132ms/step - loss: 0.6206 - accuracy: 0.7050 - recall_1: 0.9955 - precision_1: 0.7063 - val_loss: 0.9826 - val_accuracy: 0.2970 - val_recall_1: 1.0000 - val_precision_1: 0.2970
Epoch 2/50
50/50 [==============================] - 1s 27ms/step - loss: 0.6109 - accuracy: 0.7050 - recall_1: 1.0000 - precision_1: 0.7050 - val_loss: 0.9225 - val_accuracy: 0.2970 - val_recall_1: 1.0000 - val_precision_1: 0.2970
Epoch 3/50
50/50 [==============================] - 1s 27ms/step - loss: 0.6102 - accuracy: 0.7050 - recall_1: 1.0000 - precision_1: 0.7050 - val_loss: 0.8090 - val_accuracy: 0.2970 - val_recall_1: 1.0000 - val_precision_1: 0.2970
Epoch 4/50
50/50 [==============================] - 1s 27ms/step - loss: 0.5979 - accuracy: 0.7050 - recall_1: 1.0000 - precision_1: 0.7050 - val_loss: 0.7804 - val_accuracy: 0.2970 - val_recall_1: 1.0000 - val_precision_1: 0.2970
Epoch 5/50
50/50 [==============================] - 1s 28ms/step - loss: 0

50/50 [==============================] - 1s 27ms/step - loss: 0.4342 - accuracy: 0.8036 - recall_1: 0.8584 - precision_1: 0.8623 - val_loss: 0.6800 - val_accuracy: 0.6827 - val_recall_1: 0.8205 - val_precision_1: 0.4800
Epoch 37/50
50/50 [==============================] - 1s 27ms/step - loss: 0.4355 - accuracy: 0.7959 - recall_1: 0.8422 - precision_1: 0.8648 - val_loss: 0.9422 - val_accuracy: 0.5457 - val_recall_1: 0.8974 - val_precision_1: 0.3860
Epoch 38/50
50/50 [==============================] - 1s 27ms/step - loss: 0.4331 - accuracy: 0.7953 - recall_1: 0.8476 - precision_1: 0.8600 - val_loss: 0.8684 - val_accuracy: 0.6548 - val_recall_1: 0.8462 - val_precision_1: 0.4562
Epoch 39/50
50/50 [==============================] - 1s 29ms/step - loss: 0.4440 - accuracy: 0.7883 - recall_1: 0.8395 - precision_1: 0.8573 - val_loss: 0.7045 - val_accuracy: 0.6802 - val_recall_1: 0.8205 - val_precision_1: 0.4776
Epoch 40/50
50/50 [==============================] - 1s 28ms/step - loss: 0.4245 - a